# SarcGraphTools - Analysis

All demos are availble on GitHub at https://github.com/Sarc-Graph/sarcgraph/tree/main/tutorials.

To run demos with `jupyter notebook` check [Installation Guide](https://sarc-graph.readthedocs.io/en/latest/installation.html).

SarcGraph includes the tools which enable the recovery of basic sarcomere characteristics and the ability to run further high level analysis.

In this notebook we provide a tutorial on how to use the SarcGraph package using demos and examples. The focus is on the ``SarcGraphTools.Analysis`` class in the ``sg_tools`` module.

## Initialization

Methods in the ``SarcGraphTools`` class need the information that will be saved by running ``SarcGraph.sarcomere_detection()``.

To showcase this we use ``samples/sample_1.avi``.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sarcgraph.sg import SarcGraph

sg = SarcGraph(output_dir='../tutorial-results', file_type='video')
sarcomeres, _ = sg.sarcomere_detection(file_path='../samples/sample_1.avi')

sarcomeres.sample(5)

Frame 79: 91 trajectories present.


,frame,sarc_id,x,y,length,width,angle,zdiscs
4996,36,62,259.641090,159.729402,14.805879,9.987089,1.713409,"36,38"
1243,43,15,87.370502,127.381305,25.125737,24.089347,2.071055,"-30,-28"
970,10,12,71.334448,280.178625,26.141463,27.060158,1.300676,"-33,-32"
5788,28,72,134.699595,130.015240,14.941399,9.308028,3.135721,"52,76"
5289,9,66,202.904344,152.429252,11.341403,8.952443,1.150214,"44,71"


By default ``save_output=True`` in ``sg.sarcomere_detection()`` and the following information will be saved in ``../tutorial-results``:

- raw video frames (grayscale)
- filtered video frames
- zdisc contours
- segmented zdiscs information
- tracked zdiscs information
- detected sarcomeres information

The next step is to run ``SarcGraphTools.TimeSeries()`` to apply GPR on timeseries data related to each sarcomere.

In [2]:
from sarcgraph.sg_tools import SarcGraphTools

sg_tools = SarcGraphTools(input_dir='../tutorial-results')
_ = sg_tools.time_series.sarcomeres_gpr()

This step is used for noise reduction as well as interpolating missing values of the timeseries (``x_position``, ``y_position``, ``length``, ``width``, ``angle``, ``length_normalized``).

## Characterization Analysis

Here is a list of available functions in the ``SarcGraphTools.Analysis`` class:

- ``compute_F_J()``: Computes the average deformation gradient (F) and its jacobian (J) for the whole movie

- ``compute_OOP()``: Computes Orientation Order Parameter (OOP) for the whole movie

- ``compute_metrics()``: Computes (`OOP`, `C_iso`, `C_OOP`, `s_til`, `s_avg`) as defined in the [SarcGraph paper](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1009443)

- ``compute_ts_params()``: Computes (`contraction time`, `relaxation time`, `flat time`, `period`, `offset`, etc.) for timeseries

- ``create_spatial_graph()``: Generates a spatial graph of tracked z-discs where edges indicate sarcomeres and edge weights indicate the ratio of the frames in which each sarcomere is detected

**Note**

Check the api reference for [SarcGraphTools](https://sarc-graph.readthedocs.io/en/latest/reference/sg_tools.html)

<a id='analysis-functions'></a>

To run these functions, create an instance of the ``SarcGraphTools`` class and set the ``input_dir`` to the directory used previously to save the output of sarcomere detection (in this case ``input_dir='../tutorial-results'``)

In [3]:
from sarcgraph.sg_tools import SarcGraphTools

sg_tools = SarcGraphTools(input_dir='../tutorial-results')

Now, any of the functions listed [above](#analysis-functions) can be executed. For examples, run the following for ``F`` and ``J`` computation:

In [4]:
F, J = sg_tools.analysis.compute_F_J()

This also saves ``F`` and ``J`` in the specified input directory ``input_dir`` if ``save_results=True`` in ``SarcGraphTools``

Run the rest of the function:

In [5]:
OOP, OOP_vec = sg_tools.analysis.compute_OOP()

In [6]:
metrics = sg_tools.analysis.compute_metrics()

In [7]:
ts_params = sg_tools.analysis.compute_ts_params()

**Note**

``create_spatial_graph()`` needs access to the original video/image file ``file_path`` or a dataframe that contains information of detected and tracked z-discs ``tracked_zdiscs``. So, below we show two ways to run this functions:

1. Specifying ``file_path``

In [8]:
sg_tools.analysis.create_spatial_graph(file_path='../samples/sample_1.avi')

Frame 79: 91 trajectories present.


2. Specifying ``tracked_zidscs``

In [9]:
# loading saved tracked_zidscs from 'tutorial-results' folder
import pandas as pd

tracked_zdiscs = pd.read_csv(
    f"{sg_tools.input_dir}/tracked-zdiscs.csv", index_col=[0]
)

sg_tools.analysis.create_spatial_graph(tracked_zdiscs=tracked_zdiscs)

After running all functions the following files will be added to ``input_dir``:

- ``recovered_F.npy``
- ``recovered_J.npy``
- ``recovered_OOP.npy``
- ``recovered_OOP_vector.npy``
- ``recovered_metrics.json``
- ``spatial-graph.pkl``
- ``spatial-graph-pos.pkl``
- ``time_series_params.csv``